In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns

# Show plots inline
%matplotlib inline

In [2]:
# suppress future warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

In [3]:
df = pd.read_csv(r'C:\Airline data\mia_data.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Airline data\\mia_data.csv'

In [ ]:
df.sample()

In [ ]:
#These columns are not needed.

to_delete = ['ORIGIN_CITY_NAME', 'DEST_CITY_NAME', 'DEP_DEL15','DEP_DELAY_GROUP']
df.drop(to_delete, axis = 1, inplace = True)


In [ ]:
df.keys()
df.drop('QUARTER', axis = 1, inplace = True)

In [ ]:

df['WHY_DELAY']=''

In [ ]:
df.keys()

In [ ]:
#assiging value to WHY_DELAY column based on what caused the delay.

conditions = [df['CARRIER_DELAY']>0, df['WEATHER_DELAY']>0, df['NAS_DELAY']>0, df['SECURITY_DELAY']>0, df['LATE_AIRCRAFT_DELAY']>0]
choices = ['carrier', 'weather', 'nas', 'security', 'late aircraft']
df['WHY_DELAY'] = np.select(conditions, choices,)

In [ ]:
#create bar graph based on why flights were delayed.
df.WHY_DELAY.value_counts().to_frame().style.bar()

In [ ]:
df.sample()

In [ ]:
# Column FL_DATE time is wrong all times are set to midnight. I am going to get delete it to avoid further confusion.
df.drop('FL_DATE', axis = 1, inplace = True)

In [ ]:
df.keys()

In [ ]:
#Going to create DATE column by combining 'Year', 'Month', 'Day_of_Month'
df['DAY'] =df['DAY_OF_MONTH']


In [ ]:
df['DATE']=pd.to_datetime(df[['YEAR', 'MONTH','DAY']])

In [ ]:
df.drop('DAY_OF_MONTH', axis = 1, inplace = True)


In [ ]:
df.keys()

In [ ]:
#This column I want to convert to a time stamp. However, I am having problems because the data is not in the proper format.
df.CRS_ARR_TIME

In [ ]:
#This column I want to convert to a time stamp. However, I am having problems because the data is not in the proper format.
df.CRS_DEP_TIME

In [ ]:
import datetime as datetime
#_________________________________________________________
# Function that convert the 'HHMM' string to datetime.time
def format_heure(chaine):
    if pd.isnull(chaine):
        return np.nan
    else:
        if chaine == 2400: chaine = 0
        chaine = "{0:04d}".format(int(chaine))
        heure = datetime.time(int(chaine[0:2]), int(chaine[2:4]))
        return heure
#_____________________________________________________________________
# Function that combines a date and time to produce a datetime.datetime
def combine_date_heure(x):
    if pd.isnull(x[0]) or pd.isnull(x[1]):
        return np.nan
    else:
        return datetime.datetime.combine(x[0],x[1])
#_______________________________________________________________________________
# Function that combine two columns of the dataframe to create a datetime format
def create_flight_time(df, col):    
    liste = []
    for index, cols in df[['DATE', col]].iterrows():    
        if pd.isnull(cols[1]):
            liste.append(np.nan)
        elif float(cols[1]) == 2400:
            cols[0] += datetime.timedelta(days=1)
            cols[1] = datetime.time(0,0)
            liste.append(combine_date_heure(cols))
        else:
            cols[1] = format_heure(cols[1])
            liste.append(combine_date_heure(cols))
    return pd.Series(liste)


In [ ]:
df.head(10)

In [ ]:
df['DEPARTURE_TIME'] = df['CRS_DEP_TIME'].apply(format_heure)
df['ARRIVAL_TIME'] = df['CRS_ARR_TIME'].apply(format_heure)

In [4]:
df.info()

NameError: name 'df' is not defined

In [ ]:
# Create a histogram of all delayed arrival flights regardless of reason.
df['ALL_LATE_ARR'] = df.query('ARR_DELAY_NEW > 0')["CRS_ARR_TIME"]

In [ ]:
plt.hist(df.ALL_LATE_ARR)
plt.title('Frequency of Late Arrivals')
plt.xlabel('Time of Arrival')
plt.ylabel('Frequency')
plt.show()

In [ ]:
# Histogram of all delayed departure flights regardless of reason.
df['ALL_LATE_DEP'] = df.query('DEP_DELAY_NEW > 0')["CRS_DEP_TIME"]

In [ ]:
plt.hist(df.ALL_LATE_DEP)
plt.title('Frequency of Late Departures')
plt.xlabel('Time of Departure')
plt.ylabel('Frequency')
plt.show()

In [ ]:
# Calculating the average time a arrival was late by.
df_arr_mean = np.nanmean(df['ARR_DELAY_NEW'].values)

In [ ]:
df_arr_mean

In [ ]:
# Calculating the average time a departure was late by
df_dep_mean = np.nanmean(df['DEP_DELAY_NEW'])

In [ ]:
df_dep_mean

In [ ]:
# What time was the original arrival time for flights that were delayed by weather.
df['LATE_ARR'] = df.query('WHY_DELAY == "weather" & ARR_DELAY_NEW > 0')["CRS_ARR_TIME"]
df.LATE_ARR.describe()

In [ ]:
plt.hist(df.LATE_ARR)
plt.title('Frequency of Late Arrivals')
plt.xlabel('Time of Arrival')
plt.ylabel('Frequency')
plt.show()

In [ ]:
#What time was the original departure time for flights that were delayed by weather.
df['LATE_DEP'] = df.query('WHY_DELAY == "weather" & DEP_DELAY_NEW > 0')["CRS_DEP_TIME"]

In [ ]:
df.LATE_DEP.describe()

In [ ]:
plt.hist(df.LATE_DEP)
plt.title('Frequency of Late Departures')
plt.xlabel('Time of Arrival')
plt.ylabel('Frequency')
plt.show()

In [ ]:
df.LATE_ARR.unique()
#plt.scatter(x = df.LATE_ARR, y = df.LATE_ARR.value_counts())

In [ ]:
# which month of the year is the busiest for travel
df['MONTH'].value_counts()

#July is the busiest month for travel

In [ ]:
# which time of the day is the busiest
df['DEP_TIME'].value_counts()
#5:56 Am is the most common departure time.
df.head()

In [ ]:
# which carrier is the most notorious for delay
df_delayed = df[df['WHY_DELAY'] != '0']
df_delayed['OP_CARRIER'].value_counts()

In [ ]:
df['OP_CARRIER'].value_counts()

In [ ]:
print('American Airlines had more late flights than any other airline with 7260, Jet Blue had the most late flights in terms of percentage with 39.77%.')

In [ ]:
#which time of the day is more bound for cancellations.
df.keys()

In [ ]:
df_cancelled = df[df['CANCELLED'] == 1]

In [ ]:
df_cancelled['DEP_TIME'].mean()

In [ ]:
df_cancelled['DEP_TIME'].mode()

In [ ]:
df_cancelled['DEP_TIME'].median()